In [1]:
from data_utils import extract_data, load_data, preprocess_data
# extract_data("data")
data = load_data("data")
X_train, X_test, y_train, y_test = preprocess_data(data, add_post_text=True, include_pooled_comments=True, n_negative_samples_per_link=12)

/home/leo/recsys/data_utils.py:38: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html_content, "lxml")
/home/leo/recsys/data_utils.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posts_features['body'] = posts_features['body'].map(clean_html)


In [2]:
# for col in X_train.columns:
#     print(X_train[col].)
X_train['body'].iloc[0]

'For big (standard) panettoni typical values are:\n\nMould diameter of 17 cm (6.7 in).\nMould height of 11~13 cm (4.3~5.1 in).\nDough weight about 800 gr (1.7 lb).\nAlbeit those values are not defined in the Italian "standard", nor in the howto. You can also find wider and lower moulds for the same dough sizes (or upt to 1 kg / 2.2 lb). \nYou can find those measures in most webs in italian. Also, if you buy an already made panettone it will quite probably have those or very similar measures.\nIn general, you can use the formula:\n\nDough weight (gr) = 1,45 x Mould volume (cm3)\n\nFormula calculated empirically. You can check some values in this italian forum.\n\nWhen you pour the dough in the mould it will only be filled up to ¼ or ⅓. It needs to rise a lot: above the upper part of the mould. \nHere you have a Youtube video on making a homemade mould.\nAnd this page shows you how to do the mould yourself. It\'s in Spanish, but you can translate it automatically.\n\nSidenote: Orthodox s

In [3]:
X_train.drop(columns=["postid", "relatedpostid", "id", "creationdate", "postid_related",], inplace=True)
X_test.drop(columns=["postid", "relatedpostid", "id", "creationdate", "postid_related",], inplace=True)

In [4]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44428 entries, 31207 to 15795
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   linktypeid                 44428 non-null  int64  
 1   score                      44007 non-null  object 
 2   viewcount                  14226 non-null  object 
 3   answercount                14226 non-null  object 
 4   commentcount               44007 non-null  object 
 5   body                       44007 non-null  object 
 6   score_related              44362 non-null  object 
 7   viewcount_related          14519 non-null  object 
 8   answercount_related        14519 non-null  object 
 9   commentcount_related       44362 non-null  object 
 10  body_related               44362 non-null  object 
 11  vote_count                 39794 non-null  float64
 12  vote_count_related         40013 non-null  float64
 13  avg_comment_score          23840 non-null 

In [5]:
X_train.dropna().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2430 entries, 18817 to 6265
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   linktypeid                 2430 non-null   int64  
 1   score                      2430 non-null   object 
 2   viewcount                  2430 non-null   object 
 3   answercount                2430 non-null   object 
 4   commentcount               2430 non-null   object 
 5   body                       2430 non-null   object 
 6   score_related              2430 non-null   object 
 7   viewcount_related          2430 non-null   object 
 8   answercount_related        2430 non-null   object 
 9   commentcount_related       2430 non-null   object 
 10  body_related               2430 non-null   object 
 11  vote_count                 2430 non-null   float64
 12  vote_count_related         2430 non-null   float64
 13  avg_comment_score          2430 non-null   f

In [6]:
train_nan_mask = X_train.isna().any(axis=1)
X_train = X_train[~train_nan_mask]
y_train = y_train[~train_nan_mask]
test_nan_mask = X_test.isna().any(axis=1)
X_test = X_test[~test_nan_mask]
y_test = y_test[~test_nan_mask]

In [7]:
X_train["linktypeid"].value_counts()

1    1343
0    1087
Name: linktypeid, dtype: int64

In [9]:
X_test["linktypeid"].value_counts()

1    376
0    283
Name: linktypeid, dtype: int64

In [13]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2430 entries, 18817 to 6265
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   linktypeid                 2430 non-null   int64  
 1   score                      2430 non-null   object 
 2   viewcount                  2430 non-null   object 
 3   answercount                2430 non-null   object 
 4   commentcount               2430 non-null   object 
 5   body                       2430 non-null   object 
 6   score_related              2430 non-null   object 
 7   viewcount_related          2430 non-null   object 
 8   answercount_related        2430 non-null   object 
 9   commentcount_related       2430 non-null   object 
 10  body_related               2430 non-null   object 
 11  vote_count                 2430 non-null   float64
 12  vote_count_related         2430 non-null   float64
 13  avg_comment_score          2430 non-null   f

In [18]:
X_train.drop(columns=["linktypeid"], inplace=True)
X_test.drop(columns=["linktypeid"], inplace=True)

In [19]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score

# Define categorical features indices if you have any
categorical_features_indices = []  # e.g., [0, 1, 2] if your first three columns are categorical

# Initialize CatBoostClassifier
model = CatBoostClassifier(
    iterations=1000,  # Number of boosting iterations
    learning_rate=0.1,  # Learning rate
    depth=6,  # Depth of the tree
    cat_features=categorical_features_indices,  # Specify categorical features
    text_features=["body", "comments_text", "comments_text_related", "body_related"],
    verbose=100  # Print training progress every 100 iterations
)

# Train the model
model.fit(X_train, y_train, eval_set=(X_test, y_test), plot=True)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6631135	test: 0.6680372	best: 0.6680372 (0)	total: 45.4ms	remaining: 45.3s
100:	learn: 0.3417226	test: 0.5126968	best: 0.5126968 (100)	total: 6.14s	remaining: 54.6s
200:	learn: 0.2009908	test: 0.5184336	best: 0.5126968 (100)	total: 12.3s	remaining: 48.9s
300:	learn: 0.1273888	test: 0.5305683	best: 0.5126968 (100)	total: 18.5s	remaining: 42.9s
400:	learn: 0.0859770	test: 0.5462612	best: 0.5126968 (100)	total: 24.7s	remaining: 36.9s
500:	learn: 0.0614043	test: 0.5613719	best: 0.5126968 (100)	total: 31s	remaining: 30.9s
600:	learn: 0.0455839	test: 0.5784050	best: 0.5126968 (100)	total: 37.3s	remaining: 24.7s
700:	learn: 0.0345286	test: 0.5923644	best: 0.5126968 (100)	total: 43.5s	remaining: 18.6s
800:	learn: 0.0272175	test: 0.6098377	best: 0.5126968 (100)	total: 49.7s	remaining: 12.4s
900:	learn: 0.0220346	test: 0.6242904	best: 0.5126968 (100)	total: 55.9s	remaining: 6.14s
999:	learn: 0.0183160	test: 0.6404141	best: 0.5126968 (100)	total: 1m 2s	remaining: 0us

bestTest = 0.512

In [20]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,comments_text,25.322854
1,body,13.914026
2,comments_text_related,12.872172
3,score_related,11.785422
4,viewcount_related,11.120119
5,body_related,10.811885
6,vote_count_related,10.003272
7,answercount_related,1.747175
8,vote_count,0.798067
9,avg_comment_score,0.573770
